In [1]:
import tensorflow as tf
import numpy as np

In [2]:
x_data, y_data = np.load('../data/celeba/masked/x_data.npy'), np.load('../data/celeba/masked/y_data.npy')

In [50]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_gan(x_shape):
    generator = models.Sequential([
        layers.Dense(100, activation='relu', input_shape=[x_shape]),
        layers.Dense(1000, activation='relu'),
        layers.Dense(128*128*3, activation='sigmoid'),
        layers.Reshape([128, 128, 3])
    ])


    discriminator = tf.keras.models.Sequential([
        layers.Flatten(input_shape=[128,128,3]),
        layers.Dense(1000, activation='relu'),
        layers.Dense(100, activation='relu'),

        layers.Dense(1, activation='sigmoid'),
    ])

    gan = models.Sequential([generator, discriminator])

    return gan

In [51]:
z_dim = 100
gan = build_gan(z_dim)

In [52]:
generator = gan.layers[0]
discriminator = gan.layers[1]

In [53]:
generator.summary()

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_60 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_61 (Dense)             (None, 1000)              101000    
_________________________________________________________________
dense_62 (Dense)             (None, 49152)             49201152  
_________________________________________________________________
reshape_10 (Reshape)         (None, 128, 128, 3)       0         
Total params: 49,312,252
Trainable params: 49,312,252
Non-trainable params: 0
_________________________________________________________________


In [54]:
discriminator.summary()

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_9 (Flatten)          (None, 49152)             0         
_________________________________________________________________
dense_63 (Dense)             (None, 1000)              49153000  
_________________________________________________________________
dense_64 (Dense)             (None, 100)               100100    
_________________________________________________________________
dense_65 (Dense)             (None, 1)                 101       
Total params: 49,253,201
Trainable params: 49,253,201
Non-trainable params: 0
_________________________________________________________________


In [55]:
batch_size=32
x_data = x_data/255
train_ds = tf.data.Dataset.from_tensor_slices(x_data).batch(batch_size)

In [56]:
def trainGAN(dataset, batch_size, z_dim, epochs):
    gan = build_gan(z_dim)
    gene, disc = gan.layers
    
    disc.compile(loss='binary_crossentropy', optimizer='rmsprop')
    disc.trainable=False
    gan.compile(loss='binary_crossentropy', optimizer='rmsprop')
    
    
    for epoch in range(epochs):
        for ds in dataset:
            noise = tf.random.normal(shape=[batch_size, z_dim])
            generated = gene(noise)
            print(generated.shape, ds.shape)

            x_fake_and_real = tf.concat([generated, ds], axis=0)
            print(x_fake_and_real)
            y1 = tf.constant([[0.]]*batch_size +[[1.]]*batch_size)
            print(y1)
            
            disc.trainable=True
            disc.train_on_batch(x_fake_and_real, y1)

In [57]:
trainGAN(train_ds, 32, 100, 10)

(32, 128, 128, 3) (32, 128, 128, 3)


InvalidArgumentError: cannot compute ConcatV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:ConcatV2] name: concat